In [2]:
import pandas as pd
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", None)
pd.set_option("display.max_columns", None)

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, countDistinct

In [4]:
spark = SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/02/10 00:19:34 WARN Utils: Your hostname, LappyToppy, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/02/10 00:19:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/10 00:19:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/10 00:19:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
appearances = spark.read.parquet("/home/gnana/football-lakehouse/bronze/appearances")

In [6]:
appearances.printSchema()

root
 |-- appearance_id: string (nullable = true)
 |-- game_id: integer (nullable = true)
 |-- player_id: integer (nullable = true)
 |-- player_club_id: integer (nullable = true)
 |-- player_current_club_id: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- player_name: string (nullable = true)
 |-- competition_id: string (nullable = true)
 |-- yellow_cards: integer (nullable = true)
 |-- red_cards: integer (nullable = true)
 |-- goals: integer (nullable = true)
 |-- assists: integer (nullable = true)
 |-- minutes_played: integer (nullable = true)



In [7]:
appearances.limit(5).toPandas()

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
0,3869077_377641,3869077,377641,1075,162,2022-09-03,Moatasem Al-Musrati,PO1,1,0,0,0,60
1,3869077_413118,3869077,413118,1075,1075,2022-09-03,Sikou Niakaté,PO1,0,0,0,0,90
2,3869077_443338,3869077,443338,2420,2420,2022-09-03,Alisson Safira,PO1,1,0,0,0,17
3,3869077_448264,3869077,448264,2420,2420,2022-09-03,Afonso Freitas,PO1,1,0,0,0,32
4,3869077_514304,3869077,514304,2420,2420,2022-09-03,Nélson da Luz,PO1,0,0,0,0,90


In [8]:
appearances_silver = appearances.select(
    col("player_name"),
    col("appearance_id"),
    col("game_id"),
    col("player_id"),
    col("competition_id"),
    col("minutes_played"),
    col("goals"),
    col("assists"),
    col("yellow_cards"),
    col("red_cards")
).dropDuplicates(["appearance_id"])

In [9]:
appearances_silver.count()

1706806

In [10]:
appearances_silver.select("appearance_id").distinct().count()

1706806

In [11]:
appearances_silver.write.mode("overwrite").parquet("/home/gnana/football-lakehouse/silver/appearances_silver")

26/02/10 00:20:16 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                